In [ ]:
import os
import sys
import csv
import pickle
import numpy as np
import gurobipy as gp
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.io import loadmat
from time import perf_counter as timer
from concurrent.futures import ProcessPoolExecutor
from src import *

In [ ]:
data = Data()

thermal = Thermal()
data.load_thermal(thermal)

renewable = Renewable()
data.load_renewable_capacity(renewable)
data.load_renewable_generation(renewable)

demand = Demand()
data.load_demand(demand)

commitment = Commitment()
data.load_commitment_decision(commitment)

results = Results(thermal, renewable, commitment)

In [ ]:
def parallel(idx_hour):
    solver = Solver(thermal, renewable, demand, commitment, results)
    solver.solve(idx_hour)

In [ ]:
with ProcessPoolExecutor(max_workers=os.cpu_count()) as executor:
    list(executor.map(parallel, range(8760)))
results.process_outputs()

In [ ]:
# # sanity check
# # sequential
# for idx_hour in range(8760):
#     solver.solve(idx_hour)

# results.process_outputs()

# with open("sequential_results.pkl", "wb") as f:
#     pickle.dump(results, f)

# # parallel
# def parallel(idx_hour):
#     solver = Solver(thermal, renewable, demand, commitment, results)
#     solver.solve(idx_hour)

# with ProcessPoolExecutor(max_workers=20) as executor:
#     list(executor.map(parallel, range(8760)))

# results.process_outputs()

# with open("parallel_results.pkl", "wb") as f:
#     pickle.dump(results, f)

# with open("sequential_results.pkl", "rb") as f:
#     sequential_results = pickle.load(f)

# with open("parallel_results.pkl", "rb") as f:
#     parallel_results = pickle.load(f)

# print(np.all(np.array([np.all(results.smp == parallel_results.smp), np.all(results.cost_system == parallel_results.cost_system), np.all(results.p_thermal == parallel_results.p_thermal), np.all(results.p_solar == parallel_results.p_solar), np.all(results.p_wind == parallel_results.p_wind), np.all(results.p_hydro == parallel_results.p_hydro)])))
# print(np.all(np.array([np.all(results.smp == sequential_results.smp), np.all(results.cost_system == sequential_results.cost_system), np.all(results.p_thermal == sequential_results.p_thermal), np.all(results.p_solar == sequential_results.p_solar), np.all(results.p_wind == sequential_results.p_wind), np.all(results.p_hydro == sequential_results.p_hydro)])))
# print(np.all(np.array([np.all(parallel_results.smp == sequential_results.smp), np.all(parallel_results.cost_system == sequential_results.cost_system), np.all(parallel_results.p_thermal == sequential_results.p_thermal), np.all(parallel_results.p_solar == sequential_results.p_solar), np.all(parallel_results.p_wind == sequential_results.p_wind), np.all(parallel_results.p_hydro == sequential_results.p_hydro)])))

In [ ]:
# fig, axes = plt.subplots(3, 1, figsize=(30, 30))

# for ax, name, reg_generation, reg_result in zip(
#     axes,
#     ["solar", "wind", "hydro"],
#     [renewable.solar_generation, renewable.wind_generation, renewable.hydro_generation],
#     [p_solar, p_wind, p_hydro]
# ):
        

#     idx_nonzero_reg = np.where(reg_generation[idx_hour] > 0 )[0]

#     # original reg
#     ax.bar(
#         range(len(idx_nonzero_reg)),
#         reg_generation[idx_hour][idx_nonzero_reg],
#         label="Original REG Amount (Derived from KPG193)",
#     )

#     # curtailed reg
#     ax.scatter(
#         range(len(idx_nonzero_reg)),
#         np.array(model.getAttr("X", reg_result.values()))[idx_nonzero_reg],
#         # facecolor=None, edgecolor="red", 
#         color="red", marker="x",
#         label="Curtailed REG Amount (Result of ED)",
#     )

#     ymax = ax.get_ylim()[1]

#     # curtailed reg marker
#     ax.vlines(
#         x=np.where(reg_generation[idx_hour][idx_nonzero_reg] > np.array(model.getAttr("X", reg_result.values()))[idx_nonzero_reg])[0],
#         ymin=0, ymax=ymax,
#         colors="red", linestyles="--",
#         label="Is this 'originally producing' REG bus curtailed?",
#     )

#     ax.set_ylim((0, ymax))
#     ax.set_xlim((-0.5, len(idx_nonzero_reg) - 0.5))
#     ax.set_title(f"Inspection of {name} Buses with (Originally) Nonzero Production\nhour in year = {idx_hour}\n(originallly infeasible without curtailment)", fontsize=15)
#     ax.set_xlabel(f"Nonzero generation {name} buses (not bus id)", fontsize=15)
#     ax.set_ylabel("Power generation (MWh)", fontsize=15)
#     ax.legend()

# plt.show(fig)